## 🥾 Hiker Registration & Admin Tracker
A Python GUI project that aims to help adventure brands manage hiker registrations, track payments, send reminders, and view admin stats and records.

In [1]:
import os
import csv
import random
import re
import threading
from datetime import datetime
import tkinter as tk
from tkinter import Tk, ttk, Label, Entry, Radiobutton, StringVar, Button, messagebox
from tkcalendar import DateEntry
from fpdf import FPDF
import tkinter.simpledialog as simpledialog
from tkinter import Toplevel 
import pandas as pd
import smtplib
from email.message import EmailMessage
import requests  
import json

In [2]:
# Constants
CSV_FILE = "hiker_data.csv"
FUN_FACTS_FILE = "fun_facts.txt"
REMINDER_DAYS = 3
ADMIN_FILE = "admins.csv"
ADMIN_PASSWORD = "chelah123"

#Weather API Configuration
WEATHER_API_KEY = "122cf8850d43274d1968096dcb21560d"  
WEATHER_BASE_URL = "https://api.openweathermap.org/data/2.5"
#WEATHER_BASE_URL = "https://api.openweathermap.org/data/2.5/forecast?q={city name}&appid={API key}"

# Default facts if the file doesn't exist
DEFAULT_FUN_FACTS = [
    "Hiking lowers stress and improves mood.",
    "Kenya is home to over 60 mountain hiking trails.",
    "Mount Kenya is the second-highest peak in Africa.",
    "Hiking burns about 400-550 calories per hour.",
    "The word 'hike' first appeared in English in the early 1800s."
]

In [3]:
#weather class

WEATHER_API_KEY = "122cf8850d43274d1968096dcb21560d"


class WeatherService:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = WEATHER_BASE_URL
    
    def get_current_weather(self, location):
        """Get current weather for a location"""
        try:
            url = f"{self.base_url}/weather"
            params = {
                'q': location,
                'appid': self.api_key,
                'units': 'metric'
            }
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status() #raises an error for http codes
            return response.json()
        except requests.RequestException as e:
            print(f"Error fetching current weather: {e}")
            return None
    
    def get_forecast(self, location, days=5):
        """Get weather forecast for specified days"""
        try:
            url = f"{self.base_url}/forecast"
            params = {
                'q': location,
                'appid': self.api_key,
                'units': 'metric'
            }
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"Error fetching forecast: {e}")
            return None
    
    def get_weather_for_date(self, location, target_date):
        """Get weather forecast for a specific date"""
        forecast_data = self.get_forecast(location)
        if not forecast_data:
            return None
        
        # To handle both string and date object inputs
        if isinstance(target_date, str):
            target_timestamp = datetime.strptime(target_date, '%Y-%m-%d').timestamp()
        else:
            target_timestamp = target_date.timestamp()
        
        # Find the closest forecast to the target date
        closest_forecast = None
        min_diff = float('inf')
        
        for item in forecast_data['list']:
            forecast_timestamp = item['dt']
            diff = abs(forecast_timestamp - target_timestamp)
            if diff < min_diff:
                min_diff = diff
                closest_forecast = item
        
        return closest_forecast
    
    def is_weather_suitable_for_hiking(self, weather_data):
        """Determine if weather conditions are suitable for hiking"""
        if not weather_data:
            return False, "Weather data unavailable"
        
        temp = weather_data['main']['temp']
        weather_main = weather_data['weather'][0]['main'].lower()
        wind_speed = weather_data.get('wind', {}).get('speed', 0)
        
        warnings = []
        
        # Temperature checks
        if temp < 0:
            warnings.append(f"Very cold temperature: {temp}°C")
        elif temp > 35:
            warnings.append(f"Very hot temperature: {temp}°C")
        
        # Weather condition checks
        dangerous_conditions = ['thunderstorm', 'snow', 'extreme']
        if any(condition in weather_main for condition in dangerous_conditions):
            warnings.append(f"Hazardous weather: {weather_main}")
        
        # Wind speed check
        if wind_speed > 10:  # m/s
            warnings.append(f"High wind speed: {wind_speed} m/s")
        
        is_suitable = len(warnings) == 0
        warning_text = "; ".join(warnings) if warnings else "Weather conditions suitable for hiking"
        
        return is_suitable, warning_text

# Initialize weather service
weather_service = WeatherService(WEATHER_API_KEY)

In [4]:
def show_weather_popup(location, date=None):
    if date:
        weather_data = weather_service.get_weather_for_date(location, date)
    else:
        weather_data = weather_service.get_current_weather(location)

    if not weather_data:
        messagebox.showerror("Weather Error", "Could not fetch weather data.")
        return

    is_suitable, warning_message = weather_service.is_weather_suitable_for_hiking(weather_data)

    popup = Toplevel()
    popup.title(f"Weather for {location}")

    temp = weather_data['main']['temp']
    description = weather_data['weather'][0]['description'].capitalize()
    wind = weather_data.get('wind', {}).get('speed', 'N/A')

    info = f"""
    
    Temperature: {temp}°C
    Description: {description}
    Wind Speed: {wind} m/s
    Recommendation: {warning_message}
    """

    Label(popup, text=info.strip(), justify="left", padx=10, pady=10).pack()
    Button(popup, text="Close", command=popup.destroy).pack(pady=5)

    location_entry.delete(0,'end')


In [5]:
# #Weather Helper Functions 

# def format_weather_info(weather_data):
#     """Format weather data for display"""
#     if not weather_data:
#         return "Weather information unavailable"
    
#     temp = weather_data['main']['temp']
#     feels_like = weather_data['main']['feels_like']
#     humidity = weather_data['main']['humidity']
#     description = weather_data['weather'][0]['description'].title()
#     wind_speed = weather_data.get('wind', {}).get('speed', 0)
    
#     return f"""Weather Conditions:
# Temperature: {temp}°C (feels like {feels_like}°C)
# Description: {description}
# Humidity: {humidity}%
# Wind Speed: {wind_speed} m/s"""

# def show_weather_window(location, hike_date=None):
#     """Display weather information in a popup window"""
#     try:
#         weather_window = Toplevel()
#         weather_window.title(f"Weather for {location}")
#         weather_window.geometry("450x500")

#     # Create main frame with padding
#         main_frame = tk.Frame(weather_window, padx=15, pady=15)
#         main_frame.pack(fill='both', expand=True)

    # # Title
    #     title_label = tk.Label(main_frame, text=f"🌤️ Weather for {location}", font=("Arial", 14, "bold"))
    #     title_label.pack(pady=(0, 10))

    # # Create scrollable text widget
    #     text_frame = tk.Frame(weather_window, )
    #     text_frame.pack(fill='both', expand=True, padx=10, pady=10)
    
    #     text_widget = Text(text_frame, wrap='word', height=25, width=55)
    #     scrollbar = Scrollbar(text_frame, orient="vertical", command=text_widget.yview)
    #     text_widget.configure(yscrollcommand=scrollbar.set)
    
    #     text_widget.pack(side="left", fill="both", expand=True)
    #     scrollbar.pack(side="right", fill="y")

    # #loading message
    #     text_widget.insert('end', "Loading weather data...\n\n")
    #     weather_window.update()

    # # Clear loading message
    #     text_widget.delete('1.0', 'end')

    # # Header with location and time
    #     current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    #     text_widget.insert('end', f"📍 Location: {location}\n")
    #     text_widget.insert('end', f"🕒 Retrieved: {current_time}\n")
    #     text_widget.insert('end', "=" * 50 + "\n\n")

    #  # Get current weather
    #     print(f"Fetching weather for: {location}")  # print
    #     current_weather = weather_service.get_current_weather(location)
        
    #     if current_weather:
    #         print("Current weather data received")  # print
    #         text_widget.insert('end', "=== CURRENT WEATHER ===\n")
    #         text_widget.insert('end', format_weather_info(current_weather) + "\n\n")
            
    #         # Check if suitable for hiking
    #         is_suitable, warning = weather_service.is_weather_suitable_for_hiking(current_weather)
    #         if is_suitable:
    #             text_widget.insert('end', "✅ Current conditions suitable for hiking\n\n")
    #         else:
    #             text_widget.insert('end', f" Warning: {warning}\n\n")
    #     else:
    #         print("Failed to get current weather")  # Debug print
    #         text_widget.insert('end', "Unable to fetch current weather data\n\n")
        
        # # Get forecast for hike date if provided
        # if hike_date:
        #     print(f"Fetching forecast for date: {hike_date}")  # Debug print
        #     text_widget.insert('end', f"=== FORECAST FOR {hike_date} ===\n")
            
        #     # Convert hike_date to string if it's a date object
        #     if hasattr(hike_date, 'strftime'):
        #         hike_date_str = hike_date.strftime('%Y-%m-%d')
        #     else:
        #         hike_date_str = str(hike_date)
            
            # date_weather = weather_service.get_weather_for_date(location, hike_date_str)
            # if date_weather:
            #     text_widget.insert('end', format_weather_info(date_weather) + "\n\n")
                
            #     # Check suitability for hike date
            #     is_suitable, warning = weather_service.is_weather_suitable_for_hiking(date_weather)
            #     if is_suitable:
            #         text_widget.insert('end', "✅ Forecast suitable for hiking\n\n")
            #     else:
        #             text_widget.insert('end', f"⚠️ Forecast Warning: {warning}\n\n")
        #     else:
        #         text_widget.insert('end', "Forecast data not available for this date\n\n")
        
        # #Get 5-day forecast
        # forecast_data = weather_service.get_forecast(location)

        # if forecast_data:
        #     text_widget.insert('end', "=== 5-DAY FORECAST ===\n")

        #     for item in forecast_data['list'][:8]:  # Show next 8 forecasts
        #         dt = datetime.fromtimestamp(item['dt'])
        #         temp = item['main']['temp']
        #         description = item['weather'][0]['description'].title()
        #         text_widget.insert('end', f"{dt.strftime('%Y-%m-%d %H:%M')}: {temp:.1f}°C, {description}\n")
        
        # text_widget.config(state='disabled')  # Make read-only

        # # Button frame
    #     button_frame = tk.Frame(main_frame)
    #     button_frame.pack(pady=(10, 0))

    #     # Refresh button
    #     def refresh_weather():
    #         text_widget.config(state='normal')
    #         text_widget.delete('1.0', 'end')
    #         text_widget.insert('end', "🔄 Refreshing weather data...\n")
    #         weather_window.update()
    #         # Re-call the function to refresh data
    #         weather_window.destroy()
    #         show_weather_window(location, hike_date)
        
    #     tk.Button(button_frame, text="🔄 Refresh", command=refresh_weather, 
    #               bg="lightgreen", fg="black", font=("Arial", 10)).pack(side='left', padx=5)
        
    #     # Close button
    #     tk.Button(button_frame, text="❌ Close", command=weather_window.destroy, 
    #               bg="lightcoral", fg="black", font=("Arial", 10)).pack(side='left', padx=5)
        
    #     # Make window modal
    #     weather_window.transient()
    #     weather_window.grab_set()
        
    # except Exception as e:
    #     print(f"Error in show_weather_window: {e}")
    #     # Show error dialog if possible
    #     try:
    #         messagebox.showerror("Weather Error", 
    #                            f"Failed to load weather data:\n{str(e)}\n\nPlease check:\n"
    #                            "• Internet connection\n• Location spelling\n")
    #     except:
    #         print(f"Critical error: {e}")
        
    

In [6]:
def test_weather_service():
    """Test the weather service with a known location"""
    test_location = "Kiambu"
    print(f"Testing weather service with {test_location}...")
    
    current = weather_service.get_current_weather(test_location)
    if current:
        print("✅ Current weather API working")
        print(f"Temperature: {current['main']['temp']}°C")
        print(f"Description: {current['weather'][0]['description']}")
    else:
        print("❌ Current weather API failed")
    
    #forecast = weather_service.get_forecast(test_location)
    #if forecast:
        #print("✅ Forecast API working")#
        #print(f"First forecast: {forecast['list'][0]['main']['temp']}°C")
    #else:
        #print("❌ Forecast API failed")#

# test the weather service
test_weather_service()

Testing weather service with Kiambu...
✅ Current weather API working
Temperature: 15.46°C
Description: broken clouds


In [7]:
# Ensure fun facts file exists
if not os.path.exists(FUN_FACTS_FILE):
    with open(FUN_FACTS_FILE, "w", encoding="utf-8") as f:
        for fact in DEFAULT_FUN_FACTS:
            f.write(fact + "\n")

# Load fun facts
with open(FUN_FACTS_FILE, "r", encoding="utf-8") as f:
    FUN_FACTS = [line.strip() for line in f if line.strip()]


In [8]:
# Hiker class
class Hiker:
    def __init__(self, name, contact, hike, hike_date, payment, location= "", date=None):
        self.name = name
        self.contact = contact
        self.hike = hike
        self.hike_date = hike_date
        self.payment = payment
        self.location = location 
        self.date = date or datetime.today().strftime('%Y-%m-%d')

    def to_list(self):
        return [self.name, self.contact, self.hike, self.hike_date, self.payment, self.location, self.date]

#CSV_HEADERS = ["Name", "Contact", "Hike", "Hike Date", "Payment Status", "Location", "Date Registered"]

# Ensure CSV exists
CSV_FILE = "hiker_data.csv"
if not os.path.exists(CSV_FILE):
    with open(CSV_FILE, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Name", "Contact", "Hike", "Hike Date", "Payment Status", "Location", "Date Registered"])

# Contact validation
def is_valid_contact(contact):
    phone_pattern = r"^07\d{8}$"
    email_pattern = r"^[\w.-]+@[\w.-]+\.\w{2,}$"
    return re.match(phone_pattern, contact) or re.match(email_pattern, contact)

# Save registration to CSV
def save_registration(hiker):
    with open(CSV_FILE, "a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(hiker.to_list())

# Generate PDF receipt
def generate_receipt(hiker):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Hiker Registration Receipt", ln=True, align="C")
    for key, value in zip(["Name", "Contact", "Hike", "Hike Date", "Payment", "Date"], hiker.to_list()):
        pdf.cell(200, 10, txt=f"{key}: {value}", ln=True)
    file_path = f"receipt_{hiker.name.replace(' ', '_')}.pdf"
    pdf.output(file_path)
    return file_path

In [9]:
#Admin class

class Admin:
    #file_path = "admins.csv"
    file_path = ADMIN_FILE
    headers = ["ID", "Name", "Email", "Phone", "Role"]

    @classmethod
    def load_data(cls):
        if os.path.exists(cls.file_path):
            return pd.read_csv(cls.file_path)
        else:
            return pd.DataFrame(columns=cls.headers)

    @classmethod
    def save_data(cls, df):
        df.to_csv(cls.file_path, index=False)

    @classmethod
    def add_record(cls, record):
        df = cls.load_data()
        new_id = df["ID"].max() + 1 if not df.empty else 1
        record = [new_id] + record
        df.loc[len(df)] = record
        cls.save_data(df)

    @classmethod
    def delete_record(cls, record_id):
        df = cls.load_data()
        df = df[df["ID"] != record_id]
        cls.save_data(df)

    @classmethod
    def update_record(cls, record_id, updated_record):
        df = cls.load_data()
        df.loc[df["ID"] == record_id, cls.headers[1:]] = updated_record[1:]  # skip ID
        cls.save_data(df)

    @classmethod
    def search_records(cls, term):
        df = cls.load_data()
        return df[df.apply(lambda row: row.astype(str).str.contains(term, case=False).any(), axis=1)]


In [10]:
#HikerAdmin class

CSV_FILE = "hiker_data.csv"

class HikerAdmin:
    file_path = CSV_FILE
    headers = ["Name", "Contact", "Hike", "Hike Date", "Payment Status", "Location", "Date Registered"]

    @classmethod
    def load_data(cls):
        if os.path.exists(cls.file_path):
            df = pd.read_csv(cls.file_path)
            
            if "Location" not in df.columns:
                df["Location"] = ""
            return df
        else:
            return pd.DataFrame(columns=cls.headers)

    @classmethod
    def save_data(cls, df):
        df.to_csv(cls.file_path, index=False)

    @classmethod
    def add_record(cls, record):
        df = cls.load_data()
        new_id = df["ID"].max() + 1 if not df.empty else 1
        record = [new_id] + record
        df.loc[len(df)] = record
        cls.save_data(df)    

    @classmethod
    def update_record(cls, index, updated_row):
        df = cls.load_data()
        for i, value in enumerate(updated_row):
            df.iat[index, i] = value
        cls.save_data(df)

    @classmethod
    def delete_record(cls, index):
        df = cls.load_data()
        df = df.drop(index).reset_index(drop=True)
        cls.save_data(df)

    @classmethod
    def search(cls, term):
        df = cls.load_data()
        return df[df.apply(lambda row: row.astype(str).str.contains(term, case=False).any(), axis=1)]

In [11]:
# Reminder simulation
def schedule_reminder(hiker):
    def remind():
        messagebox.showinfo("Reminder", f"Reminder: {hiker.name} has a pending payment!")
    #delay = REMINDER_DAYS * 86400  # seconds
    delay = REMINDER_DAYS * 10  # seconds
    threading.Timer(delay, remind).start()

In [12]:
# Global variables for UI elements
name_entry = None
contact_entry = None
email_entry = None
hike_entry = None
hike_date_entry = None
location_entry = None
payment_var = None

In [ ]:
# Submit handler (input validation)
def submit():
    global name_entry, contact_entry, email_entry, hike_entry, hike_date_entry, payment_var, location_entry
    
    name = name_entry.get()
    contact = contact_entry.get()
    email = email_entry.get()  # Use separate email field
    hike = hike_entry.get()
    hike_date = hike_date_entry.get()
    payment = payment_var.get()
    location = location_entry.get() if location_entry else ""

    if not name or not contact or not hike or not hike_date or not payment:
        messagebox.showwarning("Input Error", "Please fill all required fields.")
        return

    # Use email for contact if provided, otherwise use contact field
    contact_info = email if email else contact
    if not is_valid_contact(contact_info):
        messagebox.showerror("Invalid Contact", "Please enter a valid phone or email.")
        return

     # Weather check if location is provided
    weather_warning_shown = False
    if location.strip():
        try:
            date_weather = weather_service.get_weather_for_date(location, hike_date)
            if date_weather:
                is_suitable, warning = weather_service.is_weather_suitable_for_hiking(date_weather)
                if not is_suitable:
                    proceed = essagebox.askyesno(
                        "Weather Warning", 
                        f"Weather warning for {hike_date} in {location}:\n\n{warning}\n\nDo you want to proceed with registration?"
                    )
                    if not proceed:
                        return
                    weather_warning_shown = True
        except Exception as e:
            print(f"Weather check failed: {e}")
      # Continue with registration even if weather check fails


    weather_btn = Button( root,  # or your frame if using one
    text="Check Weather",
    command=lambda: show_weather_popup(location_entry.get(), hike_date_entry.get())
    )
    weather_btn.pack(pady=5)  # or use .grid() if you're using grid layout

    hiker = Hiker(name, contact_info, hike, hike_date, payment, location)
    save_registration(hiker)
    receipt_path = generate_receipt(hiker)

    confirm_msg = f"{name} registered successfully.\nReceipt saved to:\n{receipt_path}"
    if payment.lower() == "partial":
        schedule_reminder(hiker)
        confirm_msg += f"\n\nReminder scheduled after {REMINDER_DAYS} days."

    fun_fact = random.choice(FUN_FACTS)
    messagebox.showinfo("✅ Registration Complete", f"{confirm_msg}\n\n💡 Hiking Fun Fact:\n{fun_fact}")

    # Clear form
    name_entry.delete(0, 'end')
    contact_entry.delete(0, 'end')
    email_entry.delete(0, 'end')
    hike_entry.delete(0, 'end')
    #location_entry.delete(0,'end')
    payment_var.set(None)

In [14]:
# Admin dashboard functions
def admin_dashboard(parent):
    dash = Toplevel(parent)
    dash.title("Admin Dashboard")
    dash.geometry("800x600")

    # Create notebook for tabs
    notebook = ttk.Notebook(dash)
    notebook.pack(fill='both', expand=True, padx=10, pady=10)

    # Admin Management Tab
    admin_frame = ttk.Frame(notebook)
    notebook.add(admin_frame, text="Admin Management")
    create_admin_management(admin_frame)

    # Hiker Management Tab
    hiker_frame = ttk.Frame(notebook)
    notebook.add(hiker_frame, text="Hiker Management")
    create_hiker_management(hiker_frame)

def create_admin_management(parent):
    search_var = tk.StringVar()

    def load_data():
        tree.delete(*tree.get_children())
        df = Admin.load_data()
        for index, row in df.iterrows():
            tree.insert('', 'end', values=list(row))

    def search():
        term = search_var.get()
        if term:
            df = Admin.search_records(term)
        else:
            df = Admin.load_data()
        tree.delete(*tree.get_children())
        for index, row in df.iterrows():
            tree.insert('', 'end', values=list(row))

    def add_new():
        add_win = Toplevel(parent)
        add_win.title("Add New Admin")
        add_win.geometry("400x300")

        entries = []
        for i, field in enumerate(Admin.headers[1:]):  # Skip ID
            tk.Label(add_win, text=field).grid(row=i, column=0, sticky='w', padx=10, pady=5)
            entry = tk.Entry(add_win, width=30)
            entry.grid(row=i, column=1, padx=10, pady=5)
            entries.append(entry)

        def save_new():
            new_data = [entry.get() for entry in entries]
            if any(val == "" for val in new_data):
                messagebox.showerror("Error", "All fields are required.")
                return
            Admin.add_record(new_data)
            add_win.destroy()
            load_data()

        tk.Button(add_win, text="Save", command=save_new).grid(row=len(entries), column=1, pady=10)

    def edit_selected():
        selected = tree.selection()
        if not selected:
            messagebox.showwarning("Select Admin", "Please select an admin record to edit.")
            return

        # Get the selected item's values
        item_values = tree.item(selected[0])['values']
        if not item_values:
            return

        admin_id = item_values[0]  # ID is the first column
        
        # Create edit window
        edit_win = Toplevel(parent)
        edit_win.title("Edit Admin Record")
        edit_win.geometry("300x300")

        entries = []
        for i, field in enumerate(Admin.headers[1:]):  # Skip ID
            tk.Label(edit_win, text=field).grid(row=i, column=0, sticky='w', padx=10, pady=5)
            entry = tk.Entry(edit_win, width=30)
            entry.insert(0, str(item_values[i + 1]))  # Fill with current values (skip ID)
            entry.grid(row=i, column=1, padx=10, pady=5)
            entries.append(entry)

        def save_changes():
            updated_data = [entry.get() for entry in entries]
            if any(val == "" for val in updated_data):
                messagebox.showerror("Error", "All fields are required.")
                return
            
            # Prepare the full record including ID
            full_record = [admin_id] + updated_data
            Admin.update_record(admin_id, full_record)
            edit_win.destroy()
            load_data()
            messagebox.showinfo("Success", "Admin record updated successfully!")

        tk.Button(edit_win, text="Save Changes", command=save_changes).grid(row=len(entries), column=1, pady=10)
        tk.Button(edit_win, text="Cancel", command=edit_win.destroy).grid(row=len(entries), column=0, pady=10)

    def delete_selected():
        selected = tree.selection()
        if not selected:
            messagebox.showwarning("Select Admin", "Please select an admin record to delete.")
            return

        # Get the selected item's values
        item_values = tree.item(selected[0])['values']
        if not item_values:
            return

        admin_id = item_values[0]  # ID is the first column
        admin_name = item_values[1]  # Name is the second column

        # Confirm deletion
        confirm = messagebox.askyesno(
            "Confirm Deletion", 
            f"Are you sure you want to delete admin '{admin_name}'?\nThis action cannot be undone."
        )
        
        if confirm:
            Admin.delete_record(admin_id)
            load_data()
            messagebox.showinfo("Success", f"Admin '{admin_name}' deleted successfully!")

    # Search frame
    search_frame = tk.Frame(parent)
    search_frame.pack(pady=10)
    tk.Label(search_frame, text="Search:").pack(side='left')
    tk.Entry(search_frame, textvariable=search_var).pack(side='left', padx=5)
    tk.Button(search_frame, text="Search", command=search).pack(side='left')

    # Treeview
    columns = Admin.headers
    tree = ttk.Treeview(parent, columns=columns, show='headings', height=15)
    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=120)
    tree.pack(pady=10, fill='both', expand=True)

    # Add scrollbar to treeview
    scrollbar = ttk.Scrollbar(parent, orient="vertical", command=tree.yview)
    tree.configure(yscrollcommand=scrollbar.set)
    scrollbar.pack(side="right", fill="y")

    # Buttons frame
    btn_frame = tk.Frame(parent)
    btn_frame.pack(pady=10)
    
    tk.Button(btn_frame, text="Add New", command=add_new, bg="green", fg="white").pack(side='left', padx=5)
    tk.Button(btn_frame, text="Edit Selected", command=edit_selected, bg="orange", fg="white").pack(side='left', padx=5)
    tk.Button(btn_frame, text="Delete Selected", command=delete_selected, bg="red", fg="white").pack(side='left', padx=5)
    tk.Button(btn_frame, text="Refresh", command=load_data, bg="blue", fg="white").pack(side='left', padx=5)

    # Double-click to edit
    def on_double_click(event):
        edit_selected()
    
    tree.bind("<Double-1>", on_double_click)

    load_data()

In [15]:
def create_hiker_management(parent):
    search_var = tk.StringVar()

    def load_data():
        tree.delete(*tree.get_children())
        df = HikerAdmin.load_data()
        for index, row in df.iterrows():
            tree.insert('', 'end', values=list(row))

    def search():
        term = search_var.get()
        if term:
            df = HikerAdmin.search(term)
        else:
            df = HikerAdmin.load_data()
        tree.delete(*tree.get_children())
        for index, row in df.iterrows():
            tree.insert('', 'end', values=list(row))

    def mark_as_paid():
        selected = tree.selection()
        if not selected:
            messagebox.showwarning("Select Hiker", "Please select a hiker record.")
            return

        # Get the selected item's values
        item_values = tree.item(selected[0])['values']
        if not item_values:
            return

        # Find the actual index in the dataframe
        df = HikerAdmin.load_data()
        hiker_name = item_values[0]  # Assuming name is first column
        
        # Find matching row
        matching_rows = df[df['Name'] == hiker_name]
        if matching_rows.empty:
            messagebox.showerror("Error", "Could not find hiker record.")
            return

        index = matching_rows.index[0]
        
        if df.loc[index, "Payment Status"].strip().lower() == "full":
            messagebox.showinfo("Already Paid", "This hiker has already completed payment.")
            return

        df.loc[index, "Payment Status"] = "Full"
        HikerAdmin.save_data(df)
        load_data()
        messagebox.showinfo("Updated", "Payment marked as Full.")

    # Search frame
    search_frame = tk.Frame(parent)
    search_frame.pack(pady=10)
    tk.Label(search_frame, text="Search:").pack(side='left')
    tk.Entry(search_frame, textvariable=search_var).pack(side='left', padx=5)
    tk.Button(search_frame, text="Search", command=search).pack(side='left')

    # Treeview
    columns = HikerAdmin.headers
    tree = ttk.Treeview(parent, columns=columns, show='headings', height=15)
    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=120)
    tree.pack(pady=10, fill='both', expand=True)

    # Buttons
    btn_frame = tk.Frame(parent)
    btn_frame.pack(pady=10)
    tk.Button(btn_frame, text="Mark as Paid", command=mark_as_paid).pack(side='left', padx=5)
    tk.Button(btn_frame, text="Refresh", command=load_data).pack(side='left', padx=5)

    load_data()

In [16]:
# Admin login
def admin_login():
    password = simpledialog.askstring("Admin Login", "Enter admin password:", show='*')
    if password == ADMIN_PASSWORD:
        admin_dashboard(root)
    else:
        messagebox.showerror("Access Denied", "Incorrect password.")

In [17]:
# Track dark mode state
dark_mode = False

# Define color themes
THEMES = {
    "light": {
        "bg": "#ffffff",
        "fg": "#000000",
        "button_bg": "#f0f0f0",
        "button_fg": "#000000"
    },
    "dark": {
        "bg": "#2e2e2e",
        "fg": "#ffffff",
        "button_bg": "#3a3a3a",
        "button_fg": "#ffffff"
    }
}

# Store references to special buttons that should keep their colors
special_buttons = []

# Toggle function
def toggle_dark_mode():
    global dark_mode
    dark_mode = not dark_mode
    theme = THEMES["dark"] if dark_mode else THEMES["light"]
    
    # Change root window background
    root.config(bg=theme["bg"])
    
    # Apply to all widgets recursively
    def apply_theme_to_widget(widget):
        try:
            widget_class = widget.winfo_class()
            
            # Skip special buttons (those with preserved colors)
            if widget in special_buttons:
                return
            
            # Apply theme based on widget type
            if widget_class in ["Label", "Frame"]:
                widget.config(bg=theme["bg"], fg=theme["fg"])
            elif widget_class == "Entry":
                if dark_mode:
                    widget.config(bg="#404040", fg=theme["fg"], insertbackground=theme["fg"])
                else:
                    widget.config(bg="white", fg=theme["fg"], insertbackground=theme["fg"])
            elif widget_class == "Button":
                widget.config(bg=theme["button_bg"], fg=theme["button_fg"])
            elif widget_class == "Radiobutton":
                widget.config(bg=theme["bg"], fg=theme["fg"], selectcolor=theme["bg"])
            
        except tk.TclError:
            pass  # Some widgets may not accept certain configurations
        
        # Recursively apply to children
        for child in widget.winfo_children():
            apply_theme_to_widget(child)
    
    apply_theme_to_widget()

In [18]:
# Main UI setup
def create_main_ui():
    global root, name_entry, contact_entry, email_entry, hike_entry, hike_date_entry, payment_var, location_entry
    root = tk.Tk()
    root.title("🥾 Hiker Registration System with Weather")
    root.geometry("500x750")

    # Title
    title_label = Label(root, text="🥾 Hiker Registration System", font=("Arial", 16, "bold"))
    title_label.pack(pady=20)

    # Form fields
    Label(root, text="Full Name *").pack()
    name_entry = Entry(root, width=40)
    name_entry.pack(pady=5)

    Label(root, text="Phone Number *").pack()
    contact_entry = Entry(root, width=40)
    contact_entry.pack(pady=5)

    Label(root, text="Email Address").pack()
    email_entry = Entry(root, width=40)
    email_entry.pack(pady=5)

    Label(root, text="Hike Name *").pack()
    hike_entry = Entry(root, width=40)
    hike_entry.pack(pady=5)

    # Location field for weather
    Label(root, text="Hike Location (for weather info)").pack()
    location_entry = Entry(root, width=40)
    location_entry.pack(pady=5)

    Label(root, text="Hike Date *").pack()
    hike_date_entry = DateEntry(root, date_pattern='yyyy-mm-dd', width=37)
    hike_date_entry.pack(pady=5)


    # # Weather check button
    # def check_weather():
    #     location = location_entry.get()
    #     hike_date = hike_date_entry.get()
    #     if location.strip():
    #         show_weather_window(location, hike_date)
    #     else:
    #         messagebox.showwarning("Location Required", "Please enter a hike location first.")

    # Button(root, text="Check Weather", command=check_weather, 
    #        bg="lightblue", fg="black", font=("Arial", 10)).pack(pady=5)
    
    Label(root, text="Payment Status *").pack()
    payment_var = StringVar()
    payment_frame = tk.Frame(root)
    payment_frame.pack(pady=5)

    Radiobutton(payment_frame, text="Full Payment", variable=payment_var, value="Full").pack(side='left')
    Radiobutton(payment_frame, text="Partial Payment", variable=payment_var, value="Partial").pack(side='left')

    # Buttons
    submit_btn = Button(root, text="Submit Registration", command=submit, bg="green", fg="white", font=("Arial", 12)).pack(pady=20)
    admin_btn = Button(root, text="Admin Login", command=admin_login, bg="blue", fg="white").pack(pady=10)
    special_buttons.append(submit_btn) 
    special_buttons.append(admin_btn)


    # Dark mode toggle
    toggle_btn = tk.Button(root, text="Dark Mode", command=toggle_dark_mode) 
    toggle_btn.pack(pady=10)
    
    # Footer
    footer_label = Label(root, text="© 2025 Hiker Registration System ", font=("Arial", 8))
    footer_label.pack(side='bottom', pady=10)

if __name__ == "__main__":
    create_main_ui()
    root.mainloop()
    

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python313\Lib\tkinter\__init__.py", line 2068, in __call__
    return self.func(*args)
           ~~~~~~~~~^^^^^^^
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_9036\1826782310.py", line 5, in admin_login
    admin_dashboard(root)
    ~~~~~~~~~~~~~~~^^^^^^
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_9036\2557468570.py", line 19, in admin_dashboard
    create_hiker_management(hiker_frame)
    ~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_9036\1597014282.py", line 73, in create_hiker_management
    load_data()
    ~~~~~~~~~^^
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_9036\1597014282.py", line 6, in load_data
    df = HikerAdmin.load_data()
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_9036\1729564477.py", line 12, in load_data
    df = pd.read_csv(cls.f